## Convolutional Layer
- http://yann.lecun.com

In [ ]:
# 필요한 라이브러리 임포트
import torch  # PyTorch 기본 라이브러리
import torch.nn as nn  # 신경망 모듈 (레이어, 손실 함수 등)
import torch.optim as optim  # 최적화 알고리즘 (SGD, Adam 등)
from torch.utils.data import DataLoader  # 데이터 배치 로딩
from torchvision import datasets, transforms  # 이미지 데이터셋 및 전처리

import matplotlib.pyplot as plt

In [ ]:
# 데이터 변환 및 로드
# 이미지 전처리 파이프라인 정의
transform = transforms.Compose([
    transforms.ToTensor(),  # PIL 이미지를 PyTorch 텐서로 변환 (0-255 → 0-1)
    transforms.Normalize((0.5,), (0.5,))  # 정규화: (픽셀 - 0.5) / 0.5 → [-1, 1] 범위로 변환
])

# FashionMNIST 학습 데이터셋 로드 (60,000개 이미지)
train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)

# FashionMNIST 검증 데이터셋 로드 (10,000개 이미지)
val_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

# 학습 데이터를 32개씩 배치로 묶어 로드 (shuffle=True: 매 에폭마다 순서 섞음)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 검증 데이터를 32개씩 배치로 묶어 로드 (shuffle=False: 순서 유지)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
# 배치 이미지 시각화 with matplotlib
import matplotlib.pyplot as plt
import numpy as np
import torchvision

# FashionMNIST 클래스 이름 정의
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# 학습 데이터에서 한 배치 가져오기
dataiter = iter(train_loader)
images, labels = next(dataiter)

# 이미지 그리드로 시각화 (8개만 표시)
num_images = 16
fig, axes = plt.subplots(4, 4, figsize=(8, 8))
axes = axes.flatten()

for i in range(num_images):
    # 정규화 역변환: [-1, 1] → [0, 1]
    img = images[i] / 2 + 0.5
    # 텐서를 넘파이 배열로 변환
    npimg = img.numpy()
    # 이미지 표시 (흑백이므로 squeeze로 채널 차원 제거)
    axes[i].imshow(npimg.squeeze(), cmap='gray')
    # 레이블 표시 (클래스 이름)
    axes[i].set_title(f'{class_names[labels[i]]}')
    # 축 숨기기
    axes[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# 1개 데이터만 자세히 확인
dataiter = iter(train_loader)
images, labels = next(dataiter)
image = images[0]
label = labels[0]
# 정규화 역변환: [-1, 1] → [0, 1]
img = image / 2 + 0.5
# 텐서를 넘파이 배열로 변환
npimg = img.numpy()
# 이미지 표시 (흑백이므로 squeeze로 채널 차원 제거)
plt.imshow(npimg.squeeze(), cmap='gray')
# 레이블 표시 (클래스 이름)
plt.title(f'Label: {class_names[label]}')
plt.axis('off')
plt.show()

In [ ]:
npimg.shape

In [ ]:
# CNN 모델 클래스 정의
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 첫 번째 합성곱 레이어: 입력 채널 1개(흑백) → 출력 채널 32개, 3x3 커널, padding=1로 크기 유지
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        
        # 두 번째 합성곱 레이어: 입력 채널 32개 → 출력 채널 64개, 3x3 커널
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        
        # MaxPooling 레이어: 2x2 윈도우에서 최댓값 선택, stride=2로 크기 절반으로 축소
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        # 첫 번째 완전연결층: 64채널 * 7x7 크기 = 3136개 입력 → 512개 출력
        self.fc1 = nn.Linear(64 * 7 * 7, 512)
        
        # 두 번째 완전연결층: 512개 입력 → 10개 클래스 출력 (FashionMNIST는 10개 카테고리)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        # Conv1 → ReLU → MaxPool: (1, 28, 28) → (32, 28, 28) → (32, 14, 14)
        x = self.pool(torch.relu(self.conv1(x)))
        
        # Conv2 → ReLU → MaxPool: (32, 14, 14) → (64, 14, 14) → (64, 7, 7)
        x = self.pool(torch.relu(self.conv2(x)))
        
        # Flatten: (64, 7, 7) → (3136) - 2D 특징맵을 1D 벡터로 펼침
        x = x.view(-1, 64 * 7 * 7)
        
        # FC1 → ReLU: (3136) → (512)
        x = torch.relu(self.fc1(x))
        
        # FC2 (출력층): (512) → (10) - 각 클래스에 대한 점수
        x = self.fc2(x)
        return x

In [ ]:
# 모델 인스턴스 생성
model = CNN()

In [ ]:
# 손실 함수: 다중 클래스 분류를 위한 Cross Entropy Loss
criterion = nn.CrossEntropyLoss()

# 최적화 알고리즘: Adam (학습률 0.001)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# 모델 훈련 함수

def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    # 지정된 에폭 수만큼 반복
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # 각 에폭마다 train과 val 단계 수행
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # 학습 모드: Dropout, BatchNorm 활성화
                loader = train_loader
            else:
                model.eval()  # 평가 모드: Dropout, BatchNorm 비활성화
                loader = val_loader

            running_loss = 0.0  # 누적 손실
            running_corrects = 0  # 누적 정답 개수

            # 배치 단위로 데이터 처리
            for inputs, labels in loader:
                # 데이터를 GPU/CPU로 이동
                inputs, labels = inputs.to(device), labels.to(device)

                # 기울기 초기화 (이전 배치의 기울기 제거)
                optimizer.zero_grad()

                # 순전파 (forward pass)
                with torch.set_grad_enabled(phase == 'train'):  # train일 때만 기울기 계산
                    outputs = model(inputs)  # 모델 예측
                    _, preds = torch.max(outputs, 1)  # 가장 높은 점수의 클래스 선택
                    loss = criterion(outputs, labels)  # 손실 계산

                    # train 단계에서만 역전파 및 가중치 업데이트
                    if phase == 'train':
                        loss.backward()  # 역전파: 기울기 계산
                        optimizer.step()  # 가중치 업데이트

                # 통계 누적
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            # 에폭 평균 손실 및 정확도 계산
            epoch_loss = running_loss / len(loader.dataset)
            epoch_acc = running_corrects.double() / len(loader.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        print()

    return model

In [ ]:
# GPU 사용 설정 (CUDA가 가능하면 GPU, 아니면 CPU 사용)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# 모델을 선택한 디바이스로 이동
model = model.to(device)

# 모델 훈련 시작 (10 에폭)
model = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10)

In [ ]:
# 모델 평가
model.eval()  # 평가 모드로 전환
correct = 0  # 정답 개수
total = 0  # 전체 샘플 개수

# 평가 시에는 기울기 계산 불필요 (메모리 절약 및 속도 향상)
with torch.no_grad():
    for inputs, labels in val_loader:
        # 데이터를 디바이스로 이동
        inputs, labels = inputs.to(device), labels.to(device)
        # 모델 예측
        outputs = model(inputs)
        # 가장 높은 점수의 클래스 선택
        _, predicted = torch.max(outputs.data, 1)
        # 전체 샘플 수 누적
        total += labels.size(0)
        # 정답 개수 누적
        correct += (predicted == labels).sum().item()

# 정확도 계산 (백분율)
accuracy = 100 * correct / total
print(f"Validation Accuracy: {accuracy:.2f}%")

# 샘플 데이터로 모델 예측 테스트
# 검증 데이터의 처음 5개 이미지 가져오기
sample_input = val_dataset.data[:5].unsqueeze(1).float().clone().detach().to(device)
# 예측값 계산 (기울기 추적 안 함)
predictions = model(sample_input).detach().cpu().numpy()
print("Predictions: ", predictions)  # 각 클래스에 대한 점수 출력
print("True labels: ", val_dataset.targets[:5].numpy())  # 실제 정답 레이블 출력

In [ ]:
for i in predictions:
    print(class_names[i.argmax()])

In [ ]:
# confusion matrix
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

all_preds = []
all_labels = []

for inputs, labels in val_loader:
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    all_preds.extend(preds.cpu().numpy())
    all_labels.extend(labels.cpu().numpy())

In [ ]:
# 혼동 행렬 계산
cm = confusion_matrix(all_labels, all_preds)

# 혼동 행렬 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')

plt.xticks(rotation=45)
plt.yticks(rotation=45)

plt.show()

In [ ]:
# report generation
from sklearn.metrics import classification_report
report = classification_report(all_labels, all_preds, target_names=class_names)
print(report)